<a href="https://colab.research.google.com/github/mohitxhkr/crypto_data_analysis-/blob/main/Ds_MohitKumarGupta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web3 Trading — Data Science Assignment (Clean Notebook)

> **Note:** Run this in **Google Colab**. It will download the datasets from the provided Google Drive IDs and generate outputs in the required folder structure.

## 1) Install dependencies

In [1]:
!pip install -q gdown pandas numpy matplotlib seaborn scikit-learn statsmodels scipy

## 2) Download or upload datasets
This cell tries to download from Google Drive using the **file IDs** shared in the assignment. If it fails (e.g., due to permissions), upload the CSVs manually into the `csv_files/` folder with the exact filenames used below.

In [2]:
import os, sys

os.makedirs("csv_files", exist_ok=True)
os.makedirs("outputs", exist_ok=True)

TRADER_ID = "1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs"   # Historical Trader Data (CSV on Drive)
SENT_ID   = "1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf"   # Fear & Greed Index (CSV on Drive)

TRADER_CSV = "csv_files/trader_data.csv"
SENT_CSV   = "csv_files/fear_greed.csv"

def try_gdown(file_id, out_path):
    try:
        import gdown
        url = f"https://drive.google.com/uc?id={file_id}"
        print(f"Downloading {out_path} from {url} ...")
        gdown.download(url, out_path, quiet=False)
        print("Download OK.")
    except Exception as e:
        print("⚠️ gdown download failed:", e)
        print(f"Please upload the file manually to: {out_path}")

# Attempt download
if not os.path.exists(TRADER_CSV):
    try_gdown(TRADER_ID, TRADER_CSV)
else:
    print("Trader CSV already present.")

if not os.path.exists(SENT_CSV):
    try_gdown(SENT_ID, SENT_CSV)
else:
    print("Sentiment CSV already present.")

Downloading...
From: https://drive.google.com/uc?id=1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs
To: /content/csv_files/trader_data.csv
100%|██████████| 47.5M/47.5M [00:00<00:00, 53.4MB/s]


Download OK.


Downloading...
From: https://drive.google.com/uc?id=1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf
To: /content/csv_files/fear_greed.csv
100%|██████████| 90.8k/90.8k [00:00<00:00, 18.0MB/s]

Download OK.


## 3) Load & quick EDA

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)

# Read CSVs
df = pd.read_csv(TRADER_CSV)
sent = pd.read_csv(SENT_CSV)

print("✅ Loaded shapes:", df.shape, sent.shape)
print("\nTrader columns:", df.columns.tolist())
print("Sentiment columns:", sent.columns.tolist())

display(df.head())
display(sent.head())

✅ Loaded shapes: (211224, 16) (2644, 4)

Trader columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']
Sentiment columns: ['timestamp', 'value', 'classification', 'date']


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


## 4) Preprocessing & Renaming
Map the provided trader columns into a consistent schema required for downstream analysis.

In [4]:
# Normalize trader column names to lower snake-like for robust mapping
def norm(s):
    return str(s).strip().lower().replace(" ", "_")

colmap_raw = {norm(c): c for c in df.columns}

# Candidate mappings from various possible headers to a canonical schema
canonical_map = {
    # canonical : [possible header variants in lower/underscored form]
    "account": ["account"],
    "symbol": ["symbol"],
    "execution_price": ["execution_price", "price", "executionprice"],
    "size": ["size", "size_usd", "qty", "quantity"],
    "side": ["side", "direction"],
    "time": ["time", "timestamp", "exec_time", "datetime"],
    "start_position": ["start_position", "startposition", "start_pos"],
    "event": ["event", "trade_event", "trade_type"],
    "closedPnL": ["closedpnl", "closed_pnl", "pnl", "realized_pnl", "closed_p&l"],
    "leverage": ["leverage", "lev"]
}

# Build mapping from actual df columns → canonical names
rename_map = {}
for canon, candidates in canonical_map.items():
    for cand in candidates:
        if cand in colmap_raw:
            rename_map[colmap_raw[cand]] = canon
            break

df = df.rename(columns=rename_map)

# Basic cleaning
if "time" in df.columns:
    df["time"] = pd.to_datetime(df["time"], errors="coerce", utc=True).dt.tz_convert(None)
    df["date"] = df["time"].dt.date
else:
    # If time missing, try to infer from index or skip date features
    df["date"] = pd.NaT

# Normalize side values if present
if "side" in df.columns:
    df["side"] = df["side"].astype(str).str.strip().str.lower().map({
        "buy": "long", "long": "long", "sell": "short", "short": "short"
    }).fillna(df["side"])

# Ensure numeric types
for col in ["execution_price", "size", "closedPnL", "leverage"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

print("After renaming/cleaning:", df.columns.tolist())
display(df.head())

After renaming/cleaning: ['account', 'Coin', 'execution_price', 'Size Tokens', 'size', 'side', 'Timestamp IST', 'start_position', 'Direction', 'closedPnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'time', 'date']


,account,Coin,execution_price,Size Tokens,size,side,Timestamp IST,start_position,Direction,closedPnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,time,date
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,long,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1970-01-01 00:28:50,1970-01-01
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,long,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1970-01-01 00:28:50,1970-01-01
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,long,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1970-01-01 00:28:50,1970-01-01
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,long,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1970-01-01 00:28:50,1970-01-01
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,long,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1970-01-01 00:28:50,1970-01-01


## 5) Aggregate daily metrics
Compute per-day trader behavior features (volume, pnl, leverage, winrate, long/short mix).

In [5]:
def aggregate_daily(df):
    if "date" not in df.columns or df["date"].isna().all():
        raise ValueError("No valid 'date' column found after parsing 'time'.")

    g = df.groupby(pd.to_datetime(df["date"]))
    daily = pd.DataFrame(index=pd.to_datetime(sorted(df["date"].dropna().unique())))
    daily.index.name = "date"

    # Trade counts & sizes
    daily["total_trades"] = g.size()
    if "size" in df.columns:
        daily["total_size"] = g["size"].sum()
        daily["avg_size"] = g["size"].mean()

    # PnL metrics
    if "closedPnL" in df.columns:
        daily["gross_pnl"] = g["closedPnL"].sum()
        daily["avg_closedPnL"] = g["closedPnL"].mean()
        daily["win_rate"] = g["closedPnL"].apply(lambda s: np.mean(s > 0))

    # Leverage
    if "leverage" in df.columns:
        daily["avg_leverage"] = g["leverage"].mean()

    # Long/short mix
    if "side" in df.columns:
        long_counts = g.apply(lambda x: (x["side"] == "long").sum())
        short_counts = g.apply(lambda x: (x["side"] == "short").sum())
        daily["long_trades"] = long_counts
        daily["short_trades"] = short_counts
        daily["long_ratio"] = (long_counts / (long_counts + short_counts)).replace([np.inf, -np.inf], np.nan)

    return daily.sort_index()

daily = aggregate_daily(df)
print("Daily shape:", daily.shape)
display(daily.head())
daily.to_csv("csv_files/daily_metrics.csv", index=True)
print("Saved: csv_files/daily_metrics.csv")

Daily shape: (1, 9)


,total_trades,total_size,avg_size,gross_pnl,avg_closedPnL,win_rate,long_trades,short_trades,long_ratio
date,,,,,,,,,
1970-01-01,211224,1.191187e+09,5639.45121,1.029696e+07,48.749001,0.411265,102696,108528,0.486195


Saved: csv_files/daily_metrics.csv


## 6) Merge with sentiment

In [6]:
# Standardize sentiment date
sent_cols = [c for c in sent.columns if str(c).strip().lower() in ["date", "day"]]
if not sent_cols:
    raise ValueError("Could not find a 'Date' column in sentiment CSV.")
sent["date"] = pd.to_datetime(sent[sent_cols[0]], errors="coerce").dt.date

# normalize classification (Fear/Greed)
cls_cols = [c for c in sent.columns if "class" in str(c).lower() or "classification" in str(c).lower()]
if not cls_cols:
    raise ValueError("Could not find a 'Classification' column in sentiment CSV.")
sent["classification"] = sent[cls_cols[0]].astype(str).str.strip().str.title().replace({
    "Extreme Fear": "Fear",
    "Extreme Greed": "Greed"
})

sent_daily = sent[["date", "classification"]].dropna()
sent_daily["date"] = pd.to_datetime(sent_daily["date"])
sent_daily = sent_daily.drop_duplicates(subset=["date"]).set_index("date").sort_index()

merged = daily.join(sent_daily, how="inner")
print("Merged shape:", merged.shape)
display(merged.head())

# Save merged
merged.to_csv("csv_files/merged_daily_with_sentiment.csv", index=True)
print("Saved: csv_files/merged_daily_with_sentiment.csv")

Merged shape: (0, 10)


,total_trades,total_size,avg_size,gross_pnl,avg_closedPnL,win_rate,long_trades,short_trades,long_ratio,classification
date,,,,,,,,,,


Saved: csv_files/merged_daily_with_sentiment.csv


## 7) Analysis & Visualization

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Boxplot of avg PnL by sentiment
df_plot = merged.dropna(subset=["classification"]).copy()
if "avg_closedPnL" in df_plot.columns and df_plot["avg_closedPnL"].notna().sum() > 0 and df_plot["classification"].nunique() >= 2:
    plt.figure(figsize=(6,4))
    sns.boxplot(x="classification", y="avg_closedPnL", data=df_plot)
    plt.title("Average ClosedPnL by Market Sentiment")
    plt.tight_layout()
    plt.savefig("outputs/boxplot_pnl_by_sentiment.png")
    plt.show()
else:
    print("⚠️ Not enough data to draw PnL boxplot.")

# Histogram of leverage by sentiment
if "avg_leverage" in df_plot.columns and df_plot["classification"].nunique() >= 2 and df_plot["avg_leverage"].notna().sum() > 0:
    plt.figure(figsize=(6,4))
    for label in df_plot["classification"].unique():
        s = df_plot.loc[df_plot["classification"] == label, "avg_leverage"].dropna()
        plt.hist(s, bins=20, alpha=0.5, label=label)
    plt.title("Distribution of Average Daily Leverage by Sentiment")
    plt.xlabel("avg_leverage"); plt.ylabel("count")
    plt.legend()
    plt.tight_layout()
    plt.savefig("outputs/hist_leverage_by_sentiment.png")
    plt.show()
else:
    print("⚠️ Not enough leverage data to plot histogram.")

# Welch's t-test: avg_closedPnL Fear vs Greed
if "avg_closedPnL" in df_plot.columns and df_plot["classification"].nunique() >= 2:
    greed = df_plot.loc[df_plot["classification"] == "Greed", "avg_closedPnL"].dropna()
    fear  = df_plot.loc[df_plot["classification"] == "Fear",  "avg_closedPnL"].dropna()
    if len(greed) >= 2 and len(fear) >= 2:
        t, p = stats.ttest_ind(greed, fear, equal_var=False)
        # Cohen's d (pooled)
        pooled_sd = np.sqrt(((greed.var(ddof=1) + fear.var(ddof=1)) / 2))
        d = (greed.mean() - fear.mean()) / pooled_sd if pooled_sd > 0 else np.nan
        print(f"Welch t-test (avg_closedPnL): t={t:.3f}, p={p:.4f}, Cohen's d={d:.3f}")
    else:
        print("⚠️ Not enough samples for t-test on avg_closedPnL.")
else:
    print("⚠️ avg_closedPnL or classification not available for t-test.")

⚠️ Not enough data to draw PnL boxplot.
⚠️ Not enough leverage data to plot histogram.
⚠️ avg_closedPnL or classification not available for t-test.


## 8) ML Baseline — Predict sentiment from prior-day trader features
To reduce leakage, we use **y = today's sentiment** and **X = previous day's features**.

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

ts = merged.copy().sort_index()
# Binary target
ts["target"] = ts["classification"].map({"Fear": 0, "Greed": 1})
# Lag features by 1 day
feature_cols = [c for c in ts.columns if c not in ["classification", "target"]]
X = ts[feature_cols].shift(1).dropna()
y = ts.loc[X.index, "target"].astype(int)

# Drop rows with any NaNs in X
X = X.dropna()
y = y.loc[X.index]

# Time-based split (80/20)
split_idx = int(0.8 * len(X))
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
    print("⚠️ Not enough class diversity for training/testing. Skipping RF.")
else:
    clf = RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred, digits=4))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

    # Feature importances
    importances = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
    display(importances.head(10))

⚠️ Not enough class diversity for training/testing. Skipping RF.


## 9) Export `ds_report.pdf`
Creates a short PDF with key figures and summary.

In [9]:
from matplotlib.backends.backend_pdf import PdfPages

pdf_path = "ds_report.pdf"
with PdfPages(pdf_path) as pdf:
    # Page 1: Title + Summary stats
    fig, ax = plt.subplots(figsize=(8.27, 11.69))  # A4 portrait in inches
    ax.axis("off")
    text = []
    text.append("Web3 Trading — DS Assignment Report\n")
    text.append(f"Generated on: {pd.Timestamp.now()}\n")
    text.append("Summary (first 5 rows of merged daily):\n")
    text.append(merged.head().to_string())
    ax.text(0.02, 0.98, "\n".join(text), va="top", ha="left", family="monospace")
    pdf.savefig(fig); plt.close(fig)

    # Page 2: PnL by Sentiment
    img1 = "outputs/boxplot_pnl_by_sentiment.png"
    if os.path.exists(img1):
        fig, ax = plt.subplots(figsize=(11.69, 8.27))  # A4 landscape
        ax.axis("off")
        ax.imshow(plt.imread(img1))
        pdf.savefig(fig); plt.close(fig)

    # Page 3: Leverage histogram
    img2 = "outputs/hist_leverage_by_sentiment.png"
    if os.path.exists(img2):
        fig, ax = plt.subplots(figsize=(11.69, 8.27))
        ax.axis("off")
        ax.imshow(plt.imread(img2))
        pdf.savefig(fig); plt.close(fig)

print(f"✅ PDF report generated at: {pdf_path}")

✅ PDF report generated at: ds_report.pdf
